In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

2023-11-26 08:13:48.775218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = "data.json"

In [3]:
def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])


    return  X, y


In [4]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [12]:
def build_model(input_shape):
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.33))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model


In [6]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

In [13]:
# create network
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = build_model(input_shape)

In [14]:
# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 257, 11, 32)       320       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 129, 6, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 129, 6, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 127, 4, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 64, 2, 32)        0         
 g2D)                                                            
                                                      

In [16]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
93/93 [==============================] - 13s 130ms/step - loss: 2.6369 - accuracy: 0.1951 - val_loss: 2.3760 - val_accuracy: 0.2140
Epoch 2/30
93/93 [==============================] - 11s 123ms/step - loss: 1.9891 - accuracy: 0.3292 - val_loss: 1.7214 - val_accuracy: 0.3876
Epoch 3/30
93/93 [==============================] - 15s 159ms/step - loss: 1.6966 - accuracy: 0.4080 - val_loss: 1.5223 - val_accuracy: 0.4778
Epoch 4/30
93/93 [==============================] - 20s 214ms/step - loss: 1.5484 - accuracy: 0.4464 - val_loss: 1.4091 - val_accuracy: 0.5155
Epoch 5/30
93/93 [==============================] - 13s 144ms/step - loss: 1.4244 - accuracy: 0.4970 - val_loss: 1.3171 - val_accuracy: 0.5316
Epoch 6/30
93/93 [==============================] - 14s 153ms/step - loss: 1.3272 - accuracy: 0.5246 - val_loss: 1.2831 - val_accuracy: 0.5599
Epoch 7/30
93/93 [==============================] - 13s 143ms/step - loss: 1.2696 - accuracy: 0.5455 - val_loss: 1.2021 - val_accuracy: 0.5855